# Is Driver Alert?
The objective of this challenge is to design a classifier that will help to decide whether the driver is alert or not. In this given dataset, there are several columns related to vehicular, environmental and driver physiological data that are acquired while driving.




In [ ]:
!pip install jovian --upgrade --quiet

In [ ]:
import jovian

In [ ]:
# Execute this to save new versions of the notebook
jovian.commit(project="alertness")

[jovian] Detected Colab notebook...
[jovian] Please enter your API key ( from https://jovian.ai/ ):
API KEY: ··········
[jovian] Uploading colab notebook to Jovian...
Committed successfully! https://jovian.ai/aurobinda005/alertness


'https://jovian.ai/aurobinda005/alertness'

In [ ]:
!pip install pandas-profiling numpy matplotlib seaborn --quiet
!pip install opendatasets scikit-learn jovian --quiet --upgrade
!pip install plotly matplotlib seaborn --upgrade --quiet
!pip install scikit-learn --upgrade --quiet
!pip install imblearn --upgrade --quiet

     |████████████████████████████████| 15.2 MB 4.2 MB/s 
     |████████████████████████████████| 11.2 MB 11.3 MB/s 
     |████████████████████████████████| 950 kB 41.4 MB/s 


Downloading Dataset.

We use opendatasets library to download data from kaggle

In [ ]:
import opendatasets as od
import pandas as pd
import numpy as np
import os

In [ ]:
od.download('https://www.kaggle.com/c/stayalert/data')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: aurobindaghosh
Your Kaggle Key: ··········


100%|██████████| 18.2M/18.2M [00:00<00:00, 48.8MB/s]



Extracting archive ./stayalert/stayalert.zip to ./stayalert


This zip file consists of 4 csv files. 'fordTrain.csv', 'fordTest.csv' are used to create train set and test dataset respectively. 'Solution.csv' is another csv file which has a column named 'Prediction', that helps us to know the accuracy or model score after comparing the prediction result coming from the test dataset. 

In [ ]:
os.listdir('stayalert')

['Solution.csv', 'fordTrain.csv', 'fordTest.csv', 'example_submission.csv']

# Data Loading

Training dataset is loaded using pandas library. There are total 604329 rows and 33 columns in the training dataset.



In [ ]:
train_set = pd.read_csv('stayalert/fordTrain.csv')
train_set

,TrialID,ObsNum,IsAlert,P1,P2,P3,P4,P5,P6,P7,...,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11
0,0,0,0,34.7406,9.84593,1400,42.8571,0.290601,572,104.8950,...,0.175,752,5.99375,0,2005,0,13.4,0,4,14.8004
1,0,1,0,34.4215,13.41120,1400,42.8571,0.290601,572,104.8950,...,0.455,752,5.99375,0,2007,0,13.4,0,4,14.7729
2,0,2,0,34.3447,15.18520,1400,42.8571,0.290601,576,104.1670,...,0.280,752,5.99375,0,2011,0,13.4,0,4,14.7736
3,0,3,0,34.3421,8.84696,1400,42.8571,0.290601,576,104.1670,...,0.070,752,5.99375,0,2015,0,13.4,0,4,14.7667
4,0,4,0,34.3322,14.69940,1400,42.8571,0.290601,576,104.1670,...,0.175,752,5.99375,0,2017,0,13.4,0,4,14.7757
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
604324,510,1194,1,32.0051,10.13240,800,75.0000,0.081731,680,88.2353,...,0.280,240,3.01875,0,1801,0,30.4,0,4,15.8113
604325,510,1195,1,32.0393,12.45040,800,75.0000,0.081731,680,88.2353,...,0.280,240,3.01875,0,1801,0,30.4,0,4,15.8018
604326,510,1196,1,32.0762,10.06180,800,75.0000,0.081731,680,88.2353,...,0.175,240,3.01875,0,1800,0,31.3,0,4,15.8120
604327,510,1197,1,32.1154,17.84500,800,75.0000,0.081731,680,88.2353,...,0.175,240,3.01875,0,1800,0,31.3,0,4,15.8270


Test data set is loaded using pandas library. There are 120840 rows and 33 columns in the test set. So the ratio of test set observations and train set observations is roughly 1:5.

In [ ]:
test_set = pd.read_csv('stayalert/fordTest.csv')
test_set

,TrialID,ObsNum,IsAlert,P1,P2,P3,P4,P5,P6,P7,...,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11
0,0,0,?,38.4294,10.94350,1000,60.0000,0.302277,508,118.1100,...,0.00,255,4.50625,0,2127,0,17.6,0,4,16.1937
1,0,1,?,38.3609,15.32120,1000,60.0000,0.302277,508,118.1100,...,0.00,255,4.50625,0,2127,0,17.6,0,4,16.1744
2,0,2,?,38.2342,11.51400,1000,60.0000,0.302277,508,118.1100,...,0.07,255,4.50625,0,2131,0,17.6,0,4,16.1602
3,0,3,?,37.9304,12.26150,1000,60.0000,0.302277,508,118.1100,...,0.07,255,4.50625,0,2131,0,17.6,0,4,16.1725
4,0,4,?,37.8085,12.36660,1000,60.0000,0.302277,504,119.0480,...,0.00,255,4.50625,0,2136,0,17.6,0,4,16.1459
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120835,99,1206,?,37.3798,17.40260,892,67.2646,0.131030,752,79.7872,...,-0.07,255,0.00000,0,658,0,0.0,0,1,16.6850
120836,99,1207,?,37.1653,5.37419,892,67.2646,0.131030,752,79.7872,...,-0.07,255,0.00000,0,657,0,0.0,0,1,16.6840
120837,99,1208,?,36.9131,9.26657,892,67.2646,0.131030,752,79.7872,...,-0.07,255,0.00000,0,655,0,0.0,0,1,16.6487
120838,99,1209,?,36.6297,10.41710,892,67.2646,0.131030,752,79.7872,...,-0.07,240,0.00000,0,657,0,0.0,0,1,16.6314


In test and train dataset all the columns have either integer or float data type. So, all the columns are numerical which implies there is no categorical features. In our target column, 'IsAlert', we have only two values 0 and 1. 0 means the driver is not alert and 1 signifies the driver is alert.

In [ ]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 604329 entries, 0 to 604328
Data columns (total 33 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   TrialID  604329 non-null  int64  
 1   ObsNum   604329 non-null  int64  
 2   IsAlert  604329 non-null  int64  
 3   P1       604329 non-null  float64
 4   P2       604329 non-null  float64
 5   P3       604329 non-null  int64  
 6   P4       604329 non-null  float64
 7   P5       604329 non-null  float64
 8   P6       604329 non-null  int64  
 9   P7       604329 non-null  float64
 10  P8       604329 non-null  int64  
 11  E1       604329 non-null  float64
 12  E2       604329 non-null  float64
 13  E3       604329 non-null  int64  
 14  E4       604329 non-null  int64  
 15  E5       604329 non-null  float64
 16  E6       604329 non-null  int64  
 17  E7       604329 non-null  int64  
 18  E8       604329 non-null  int64  
 19  E9       604329 non-null  int64  
 20  E10      604329 non-null  

In [ ]:
train_set.describe()


,TrialID,ObsNum,IsAlert,P1,P2,P3,P4,P5,P6,P7,...,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11
count,604329.000000,604329.000000,604329.000000,604329.000000,604329.000000,604329.000000,604329.000000,604329.000000,604329.000000,604329.000000,...,604329.000000,604329.000000,604329.000000,604329.000000,604329.000000,604329.0,604329.000000,604329.0,604329.000000,604329.000000
mean,250.167657,603.841765,0.578799,35.449020,11.996525,1026.671035,64.061965,0.178923,845.384610,77.887628,...,-0.037710,573.786433,19.961030,0.179814,1715.688383,0.0,12.710354,0.0,3.312257,11.668277
std,145.446164,348.931601,0.493752,7.484629,3.760292,309.277877,19.755950,0.372309,2505.335141,18.577930,...,0.403896,298.412888,63.269456,0.384033,618.176470,0.0,11.532085,0.0,1.243586,9.934423
min,0.000000,0.000000,0.000000,-22.481200,-45.629200,504.000000,23.885300,0.038920,128.000000,0.262224,...,-4.795000,240.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1.000000,1.676730
25%,125.000000,302.000000,0.000000,31.758100,9.903540,792.000000,49.180300,0.092110,668.000000,66.666700,...,-0.175000,255.000000,1.487500,0.000000,1259.000000,0.0,0.000000,0.0,3.000000,7.947680
50%,250.000000,604.000000,1.000000,34.145100,11.400400,1000.000000,60.000000,0.105083,800.000000,75.000000,...,0.000000,511.000000,3.018750,0.000000,1994.000000,0.0,12.800000,0.0,4.000000,10.772600
75%,374.000000,906.000000,1.000000,37.311900,13.644200,1220.000000,75.757600,0.138814,900.000000,89.820400,...,0.070000,767.000000,7.481250,0.000000,2146.000000,0.0,21.900000,0.0,4.000000,15.270900
max,510.000000,1210.000000,1.000000,101.351000,71.173700,2512.000000,119.048000,27.202200,228812.000000,468.750000,...,3.990000,1023.000000,484.488000,1.000000,4892.000000,0.0,82.100000,0.0,7.000000,262.534000


In [ ]:
train_set.isna().sum()

TrialID    0
ObsNum     0
IsAlert    0
P1         0
P2         0
P3         0
P4         0
P5         0
P6         0
P7         0
P8         0
E1         0
E2         0
E3         0
E4         0
E5         0
E6         0
E7         0
E8         0
E9         0
E10        0
E11        0
V1         0
V2         0
V3         0
V4         0
V5         0
V6         0
V7         0
V8         0
V9         0
V10        0
V11        0
dtype: int64

We observe that there is no missing value in our training data.

# Exploratory Data Analysis

It is always a good practice to figure out the relation between different feature columns with the target columns visually. To do that we use several libraries such as matplotlib, plotly, and seaborn.

In [ ]:
import plotly.express as px
import matplotlib 
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
fig = px.histogram(train_set,x = "IsAlert")
fig.update_layout(bargap = 0.2)
fig.show()

No. of not alert observation is around 250000 and no of alert drivers are around 350000. So the difference between the number of observations of these two labels is not very significant.

In the next block, we try to find which psychological feature has the maximum mean. We obtain P3 has the highest mean.

In [ ]:
## Which psychological feature has the maximum mean
mean_val = {}
for i in train_set.columns[3:10]:
  mean_val[i] = train_set[i].mean()

max_val = sorted(mean_val.values(),reverse = True)[0]
for key,val in mean_val.items():
  if(val==max_val):
    print(f"column name: {key} and mean value = {val}")
    break


column name: P3 and mean value = 1026.6710351480733


In the next block, we try to find which environmental feature has the maximum mean. It is observed that E6 has the highest mean value.

In [ ]:
## Which environmental feature has the maximum mean
mean_val = {}
for i in train_set.columns[11:22]:
  mean_val[i] = train_set[i].mean()

max_val = sorted(mean_val.values(),reverse = True)[0]
for key,val in mean_val.items():
  if(val==max_val):
    print(f"column name: {key} and mean value = {val}")
    break

column name: E6 and mean value = 358.6747384289021


Finally, in the next block, we figure out that V6 has the maximum mean amoung the vehicular feature columns.

In [ ]:
## Which vehicular feature has the maximum mean
mean_val = {}
for i in train_set.columns[22:]:
  mean_val[i] = train_set[i].mean()

max_val = sorted(mean_val.values(),reverse = True)[0]
for key,val in mean_val.items():
  if(val==max_val):
    print(f"column name: {key} and mean value = {val}")
    break

column name: V6 and mean value = 1715.6883833143868


The environmental feature E6 column has maximum counts in between 320 and 410. For E6=366 and 367 we observe the maximum counts. In this region, IsAlert = 0 count is more than IsAlert = 1 count.

In [ ]:
fig2 = px.histogram(train_set,x = "E6", color = 'IsAlert', title = "Environmental Feature Column (E6) which has highest mean among all the environmental columns.")
fig2.update_layout(bargap = 0.1)
fig2.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
fig2 = px.histogram(train_set,x = "P3", color = 'IsAlert', title = "Psychological Feature Column (P3) which has highest mean among all the pyschological feature columns.")
fig2.update_layout(bargap = 0.1)
fig2.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
fig4 = px.histogram(train_set,x = "V6", color = 'IsAlert', title = "Vehicular Feature Column (V6) which has highest mean among all the vehicular columns.")
fig4.update_layout(bargap = 0.1)
fig4.show()

Output hidden; open in https://colab.research.google.com to view.

 Thecorrelation between IsAlert and E6 columns is -0.19.

In [ ]:
train_set.IsAlert.corr(train_set.E6)

-0.18919786734413285

The correlation between IsAlert and V6 columns is -0.24.

In [ ]:
train_set.IsAlert.corr(train_set.V6)

-0.244150113935766

The correlation between P3 and IsAlert is 0.005.

In [ ]:

train_set.IsAlert.corr(train_set.P3)

0.005167867220705532

We observe, V6 has the stronger correlation with IsAlert column than E6 and P3 but the correlation value is negative.

In [ ]:
jovian.commit()

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
Committed successfully! https://jovian.ai/aurobinda005/alertness


'https://jovian.ai/aurobinda005/alertness'

# Feature Engineering

In [ ]:
train_set.head(5)

,TrialID,ObsNum,IsAlert,P1,P2,P3,P4,P5,P6,P7,...,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11
0,0,0,0,34.7406,9.84593,1400,42.8571,0.290601,572,104.895,...,0.175,752,5.99375,0,2005,0,13.4,0,4,14.8004
1,0,1,0,34.4215,13.41120,1400,42.8571,0.290601,572,104.895,...,0.455,752,5.99375,0,2007,0,13.4,0,4,14.7729
2,0,2,0,34.3447,15.18520,1400,42.8571,0.290601,576,104.167,...,0.280,752,5.99375,0,2011,0,13.4,0,4,14.7736
3,0,3,0,34.3421,8.84696,1400,42.8571,0.290601,576,104.167,...,0.070,752,5.99375,0,2015,0,13.4,0,4,14.7667
4,0,4,0,34.3322,14.69940,1400,42.8571,0.290601,576,104.167,...,0.175,752,5.99375,0,2017,0,13.4,0,4,14.7757


In our analysis, we do not require TrialID, and ObsNum columns. On the other hand, as mentioned in Kaggle site that we should use less no of psychological features. So, we drop all the psychological features but P3 because it has the maximum mean among all the psychological columns.

In [ ]:
train_set.drop(columns = ['TrialID','ObsNum','P1','P2','P4','P5','P6','P7','P8'],inplace = True)
test_set.drop(columns = ['TrialID','ObsNum','P1','P2','P4','P5','P6','P7','P8'],inplace = True)

In [ ]:
train_set.head(5)

,IsAlert,P3,E1,E2,E3,E4,E5,E6,E7,E8,...,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11
0,0,1400,0.0,0.0,1,-20,0.015875,324,1,1,...,0.175,752,5.99375,0,2005,0,13.4,0,4,14.8004
1,0,1400,0.0,0.0,1,-20,0.015875,324,1,1,...,0.455,752,5.99375,0,2007,0,13.4,0,4,14.7729
2,0,1400,0.0,0.0,1,-20,0.015875,324,1,1,...,0.280,752,5.99375,0,2011,0,13.4,0,4,14.7736
3,0,1400,0.0,0.0,1,-20,0.015875,324,1,1,...,0.070,752,5.99375,0,2015,0,13.4,0,4,14.7667
4,0,1400,0.0,0.0,1,-20,0.015875,324,1,1,...,0.175,752,5.99375,0,2017,0,13.4,0,4,14.7757


In [ ]:
train_set.describe()

,IsAlert,P3,E1,E2,E3,E4,E5,E6,E7,E8,...,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11
count,604329.000000,604329.000000,604329.000000,604329.000000,604329.000000,604329.000000,604329.000000,604329.000000,604329.000000,604329.000000,...,604329.000000,604329.000000,604329.000000,604329.000000,604329.000000,604329.0,604329.000000,604329.0,604329.000000,604329.000000
mean,0.578799,1026.671035,10.512332,102.790045,0.290565,-4.230136,0.016262,358.674738,1.757296,1.383058,...,-0.037710,573.786433,19.961030,0.179814,1715.688383,0.0,12.710354,0.0,3.312257,11.668277
std,0.493752,309.277877,14.049071,127.258629,1.006162,35.508596,0.002304,27.399973,2.854852,1.608807,...,0.403896,298.412888,63.269456,0.384033,618.176470,0.0,11.532085,0.0,1.243586,9.934423
min,0.000000,504.000000,0.000000,0.000000,0.000000,-250.000000,0.008000,260.000000,0.000000,0.000000,...,-4.795000,240.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1.000000,1.676730
25%,0.000000,792.000000,0.000000,0.000000,0.000000,-8.000000,0.015686,348.000000,0.000000,0.000000,...,-0.175000,255.000000,1.487500,0.000000,1259.000000,0.0,0.000000,0.0,3.000000,7.947680
50%,1.000000,1000.000000,0.000000,0.000000,0.000000,0.000000,0.016001,365.000000,1.000000,1.000000,...,0.000000,511.000000,3.018750,0.000000,1994.000000,0.0,12.800000,0.0,4.000000,10.772600
75%,1.000000,1220.000000,28.240000,211.584000,0.000000,6.000000,0.016694,367.000000,2.000000,2.000000,...,0.070000,767.000000,7.481250,0.000000,2146.000000,0.0,21.900000,0.0,4.000000,15.270900
max,1.000000,2512.000000,243.991000,359.995000,4.000000,260.000000,0.023939,513.000000,25.000000,9.000000,...,3.990000,1023.000000,484.488000,1.000000,4892.000000,0.0,82.100000,0.0,7.000000,262.534000


Scalling all the values of the input columns in between -1 and +1 by using MinMaxScaler 

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [ ]:
numeric_column = train_set.columns.tolist()[1:]
print(numeric_column)

['P3', 'E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'E10', 'E11', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11']


In [ ]:
scaler.fit(train_set[numeric_column])

MinMaxScaler()

In [ ]:
train_set[numeric_column] = scaler.transform(train_set[numeric_column])
test_set[numeric_column] = scaler.transform(test_set[numeric_column])


In [ ]:
jovian.commit()

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
Committed successfully! https://jovian.ai/aurobinda005/alertness


'https://jovian.ai/aurobinda005/alertness'

In [ ]:
y_test = test_set['IsAlert']

In [ ]:
test_set.drop(columns = ['IsAlert'],inplace = True)

In [ ]:
test_set.sample(5)

,P3,E1,E2,E3,E4,E5,E6,E7,E8,E9,...,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11
119840,0.047809,0.000131,0.096854,0.0,0.490196,0.498024,0.418972,0.00,0.000000,1.0,...,0.537849,0.000000,0.000000,0.0,0.132870,0.0,0.000000,0.0,0.0,0.059075
4417,0.354582,0.107410,0.001478,0.0,0.458824,0.205534,0.201581,0.04,0.333333,1.0,...,0.545817,0.673052,0.000000,0.0,0.317253,0.0,0.000000,0.0,0.5,0.040585
25085,0.089641,0.078532,0.648215,0.0,0.533333,0.501976,0.363636,0.28,0.444444,0.0,...,0.553785,0.673052,0.009301,0.0,0.424162,0.0,0.235079,0.0,0.5,0.044311
38473,0.071713,0.000000,0.000000,0.0,0.490196,0.498024,0.083004,0.04,0.333333,1.0,...,0.589641,0.980843,0.012371,0.0,0.448078,0.0,0.306943,0.0,0.5,0.043065
84606,0.274900,0.128640,0.680132,0.0,0.490196,0.391304,0.545455,0.00,0.222222,1.0,...,0.545817,0.326948,0.003070,0.0,0.425388,0.0,0.248477,0.0,0.5,0.053724


In [ ]:
#test_set.drop(columns = test_set.columns[0:2],inplace = True)

In [ ]:
test_set.sample(5)

,P3,E1,E2,E3,E4,E5,E6,E7,E8,E9,...,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11
30222,0.163347,0.116738,0.679207,0.0,0.501961,0.494071,0.162055,0.04,0.333333,1.0,...,0.577689,0.326948,0.006231,0.0,0.399836,0.0,0.140073,0.0,0.5,0.042982
64504,0.043825,0.000000,0.000000,0.0,0.462745,0.624506,0.130435,0.04,0.333333,0.0,...,0.525896,0.673052,0.003070,1.0,0.409035,0.0,0.210719,0.0,0.5,0.041755
65868,0.358566,0.000000,0.000000,0.0,0.537255,0.592885,0.209486,0.04,0.333333,1.0,...,0.577689,0.653895,0.009301,0.0,0.389411,0.0,0.020706,0.0,0.5,0.044697
66961,0.207171,0.095807,0.946566,0.0,0.317647,0.581028,0.221344,0.04,0.222222,1.0,...,0.565737,0.326948,0.006231,0.0,0.287817,0.0,0.372716,0.0,0.5,0.043161
104940,0.055777,0.000000,0.000000,0.0,0.498039,0.735178,0.252964,0.04,0.333333,0.0,...,0.553785,1.000000,0.003070,0.0,0.446239,0.0,0.000000,0.0,0.5,0.044648


In [ ]:
y_test

0         ?
1         ?
2         ?
3         ?
4         ?
         ..
120835    ?
120836    ?
120837    ?
120838    ?
120839    ?
Name: IsAlert, Length: 120840, dtype: object

In [ ]:
y_train = train_set['IsAlert']

In [ ]:
 train_set.drop(columns = ['IsAlert'],inplace = True)

In [ ]:
train_set.sample(3)

,P3,E1,E2,E3,E4,E5,E6,E7,E8,E9,...,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11
218834,0.288845,0.000000,0.000000,0.0,0.490196,0.525692,0.418972,0.00,0.000000,1.0,...,0.494024,0.019157,0.003070,1.0,0.427433,0.0,0.248477,0.0,0.5,0.044170
149727,0.103586,0.131325,0.946927,0.0,0.490196,0.490119,0.328063,0.04,0.222222,1.0,...,0.565737,0.653895,0.009301,0.0,0.409648,0.0,0.000000,0.0,0.5,0.023815
534559,0.179283,0.000000,0.000000,0.0,0.286275,0.470356,0.501976,0.60,0.888889,0.0,...,0.513944,0.653895,0.003070,0.0,0.451145,0.0,0.208283,0.0,0.5,0.037810



# Train Validation Set Splitting




The train data is now splitted into another train data and validation set. The ratio of the new train set and validation set is 80:20.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, train_target, val_target = train_test_split(train_set,y_train,random_state = 42,test_size = 0.2)
X_train_1= X_train
X_val_1 = X_val
train_target_1 = train_target
val_target_1 = val_target

In [ ]:
X_train.shape

(483463, 23)

In [ ]:
X_val.shape

(120866, 23)

Since we have very large training sample as well as validation set data, it will take more time to implement ML algorithms on these huge data sets. So, we can take the help of sampling. We randomly select 60000 observations from training set and 25000 data from validation set.

In [ ]:
train_set_raw = X_train.join(train_target) 

In [ ]:
train_sample = train_set_raw.sample(60000)

In [ ]:
train_sample.head(5)

,P3,E1,E2,E3,E4,E5,E6,E7,E8,E9,...,V3,V4,V5,V6,V7,V8,V9,V10,V11,IsAlert
603363,0.039841,0.000000,0.000000,0.0,0.517647,0.553360,0.537549,0.04,0.222222,1.0,...,0.653895,0.006231,0.0,0.349346,0.0,0.255786,0.0,0.5,0.055497,1
93500,0.262948,0.098549,0.908315,0.0,0.482353,0.490119,0.407115,0.04,0.111111,1.0,...,0.346105,0.000000,0.0,0.330131,0.0,0.000000,0.0,0.5,0.024021,1
221126,0.296813,0.000000,0.000000,0.0,0.490196,0.371542,0.422925,0.00,0.000000,1.0,...,0.673052,0.037114,0.0,0.131848,0.0,0.000000,0.0,0.0,0.045450,1
434529,0.141434,0.119156,0.302765,0.0,0.501961,0.478261,0.047431,0.04,0.222222,1.0,...,1.000000,0.000000,0.0,0.419256,0.0,0.375152,0.0,0.5,0.043448,1
91931,0.109562,0.000000,0.000000,0.0,0.490196,0.205534,0.426877,0.08,0.333333,1.0,...,0.000000,0.136265,1.0,0.128577,0.0,0.000000,0.0,0.0,0.020047,1


In [ ]:
y_train = train_sample['IsAlert']
X_train = train_sample[train_sample.columns.tolist()[0:-1]]

In [ ]:
X_train

,P3,E1,E2,E3,E4,E5,E6,E7,E8,E9,...,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11
603363,0.039841,0.000000,0.000000,0.0,0.517647,0.553360,0.537549,0.04,0.222222,1.0,...,0.553785,0.653895,0.006231,0.0,0.349346,0.0,0.255786,0.0,0.5,0.055497
93500,0.262948,0.098549,0.908315,0.0,0.482353,0.490119,0.407115,0.04,0.111111,1.0,...,0.525896,0.346105,0.000000,0.0,0.330131,0.0,0.000000,0.0,0.5,0.024021
221126,0.296813,0.000000,0.000000,0.0,0.490196,0.371542,0.422925,0.00,0.000000,1.0,...,0.482072,0.673052,0.037114,0.0,0.131848,0.0,0.000000,0.0,0.0,0.045450
434529,0.141434,0.119156,0.302765,0.0,0.501961,0.478261,0.047431,0.04,0.222222,1.0,...,0.513944,1.000000,0.000000,0.0,0.419256,0.0,0.375152,0.0,0.5,0.043448
91931,0.109562,0.000000,0.000000,0.0,0.490196,0.205534,0.426877,0.08,0.333333,1.0,...,0.525896,0.000000,0.136265,1.0,0.128577,0.0,0.000000,0.0,0.0,0.020047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381703,0.310757,0.000000,0.000000,0.0,0.545098,0.747036,0.418972,0.04,0.333333,1.0,...,0.513944,0.346105,0.006231,1.0,0.382870,0.0,0.000000,0.0,0.5,0.023522
134801,0.346614,0.000000,0.000000,1.0,0.490196,0.664032,0.347826,0.00,0.000000,1.0,...,0.346614,0.673052,0.043345,1.0,0.478945,0.0,0.285018,0.0,0.5,0.051494
247202,0.396414,0.125025,0.513979,0.0,0.458824,0.537549,0.339921,0.04,0.111111,1.0,...,0.545817,0.019157,0.003070,0.0,0.470360,0.0,0.205847,0.0,0.5,0.059307
497588,0.047809,0.000000,0.000000,1.0,0.505882,0.521739,0.335968,0.00,0.000000,1.0,...,0.545817,0.019157,0.012371,0.0,0.417416,0.0,0.404385,0.0,0.5,0.054400


In [ ]:
val_set_raw = X_val.join(val_target) 
val_sample = val_set_raw.sample(25000)
y_val = val_sample['IsAlert']
X_val = val_sample[val_sample.columns.tolist()[0:-1]]

# Training with Logistic Regression

Logistic regression is a very useful ML algorithm to solve binary classification problems. In our challenge also, it is basically a binary classification problem because the Target column, 'IsAlert' has only 2 responses i.e 0 and 1 ,where 0 represents that the driver is not alert and 1 represents driver is alert.

Importing required libraries

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model_1 = LogisticRegression(solver = 'liblinear')

In [ ]:
model_1.fit(X_train,y_train)

LogisticRegression(solver='liblinear')

weights to the individual columns are listed below.

In [ ]:
model_1.coef_

array([[-3.22562164e-03,  1.61678492e+00, -3.62143066e-01,
         1.12532095e+00, -1.61418644e-01, -1.12705178e+00,
        -3.34046322e+00, -1.11181668e+00, -1.87689644e+00,
         3.09773251e+00,  2.20855848e+00,  1.79455941e-01,
        -1.90567717e+00, -3.16249875e-01, -2.73055387e-01,
        -7.66364782e-02, -7.06551866e-02,  1.72690412e+00,
         0.00000000e+00, -3.97969131e-01,  0.00000000e+00,
        -1.11715446e+00,  3.21057135e+01]])

intercept value is given by,

In [ ]:
model_1.intercept_

array([-1.10493875])

In [ ]:
train_preds = model_1.predict(X_train)

In [ ]:
from sklearn.metrics import accuracy_score,classification_report
print("train accuracy: ",accuracy_score(train_preds, y_train))
print("train score: ",model_1.score(X_train, y_train))

train accuracy:  0.7945666666666666
train score:  0.7945666666666666


In [ ]:
val_preds = model_1.predict(X_val)

In [ ]:
print("validation accuracy: ",accuracy_score(val_preds, y_val))
print("validation score: ",model_1.score(X_val, y_val))

validation accuracy:  0.7918
validation score:  0.7918


The accuracy of training set and validation set are not bad before performing hyperparameter tuning.

# Hyperparameter Tuning for Logistic Regression Model

We use max_iter, and solver as our hyperparameters.

In [ ]:
max_iter = [20,30,50,70,80,90,100,120,140,160,180,200,500]
solver = ['newton-cg', 'sag', 'saga','lbfgs']

In [ ]:
for i in solver:
  model_h = LogisticRegression(solver = i,n_jobs = -1)
  model_h.fit(X_train,y_train)
  train_preds = model_h.predict(X_train)
  print(f"Solver: {i}, train error:  ",1-model_h.score(X_train,y_train),end = " ")
  val_preds = model_h.predict(X_val)
  print("validation set error: ",1-model_h.score(X_val,y_val) )


Solver: newton-cg, train error:   0.20535000000000003 validation set error:  0.20811999999999997
Solver: sag, train error:   0.20528333333333337 validation set error:  0.20831999999999995
Solver: saga, train error:   0.20533333333333337 validation set error:  0.20816
Solver: lbfgs, train error:   0.20525000000000004 validation set error:  0.20808000000000004


For solver = 'sag', we get highest accuracy score for validation set.

In [ ]:
for i in max_iter:
  model_h = LogisticRegression(solver = 'sag',n_jobs = -1,max_iter = i)
  model_h.fit(X_train,y_train)
  train_preds = model_h.predict(X_train)
  print(f"max_iter: {i}, train error:  ",1-model_h.score(X_train,y_train),end = " ")
  val_preds = model_h.predict(X_val)
  print("validation set error: ",1-model_h.score(X_val,y_val) )


max_iter: 20, train error:   0.20533333333333337 validation set error:  0.20820000000000005
max_iter: 30, train error:   0.2053666666666667 validation set error:  0.20820000000000005
max_iter: 50, train error:   0.20528333333333337 validation set error:  0.20799999999999996
max_iter: 70, train error:   0.20530000000000004 validation set error:  0.20820000000000005
max_iter: 80, train error:   0.20530000000000004 validation set error:  0.20816
max_iter: 90, train error:   0.2052666666666667 validation set error:  0.20828000000000002
max_iter: 100, train error:   0.20525000000000004 validation set error:  0.20811999999999997
max_iter: 120, train error:   0.20533333333333337 validation set error:  0.20799999999999996
max_iter: 140, train error:   0.2052666666666667 validation set error:  0.20808000000000004
max_iter: 160, train error:   0.20530000000000004 validation set error:  0.20799999999999996
max_iter: 180, train error:   0.20535000000000003 validation set error:  0.2080800000000000

In [ ]:
model_h = LogisticRegression(solver = 'sag',n_jobs = -1,max_iter = 50)
model_h.fit(X_train,y_train)
val_preds = model_h.predict(X_val)
print(f"Validation set accuracy score:  ",accuracy_score(val_preds,y_val))

Validation set accuracy score:   0.79188


So we get accuracy score for validation set = 79.2%


In [ ]:
jovian.commit()

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
Committed successfully! https://jovian.ai/aurobinda005/alertness


'https://jovian.ai/aurobinda005/alertness'

# Prediction on Test Set using Optimized Logistic Regression Model

In [ ]:
model_test = LogisticRegression(solver = 'sag',n_jobs = -1,max_iter = 50)

In [ ]:
model_test.fit(X_train,y_train)

LogisticRegression(max_iter=50, n_jobs=-1, solver='sag')

In [ ]:
predict_test = model_test.predict(test_set)

In [ ]:
sol_df = pd.read_csv("stayalert/Solution.csv")

accuracy_score(predict_test,sol_df['Prediction'])

0.8406570671962926

We achieve our accuracy score on the test set 0.84, which is pretty decent to classify the drivers' alertness based on different types of data.

# Training with Random Forest Model


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
tree_model = RandomForestClassifier(random_state = 42, n_jobs = -1)
tree_model.fit(X_train,y_train)
train_tree = tree_model.predict(X_train)
print("train score:  ",tree_model.score(X_train,y_train),end = " ")
val_tree = tree_model.predict(X_val)
print(",  validation set score: ",tree_model.score(X_val,y_val) )

train score:   1.0 ,  validation set score:  0.95784


We achieve train sore 100% and validation score 95.7%, which are very good.

The probability of predictions are printed below.

In [ ]:
train_probability = tree_model.predict_proba(X_train)
train_probability

array([[0.05, 0.95],
       [0.14, 0.86],
       [0.  , 1.  ],
       ...,
       [0.03, 0.97],
       [0.01, 0.99],
       [0.72, 0.28]])

In [ ]:
#from sklearn.tree import plot_tree, export_text
#plot_tree(tree_model, feature_names=X_train.columns, max_depth=2, filled=True);

In [ ]:
importance_df = pd.DataFrame({
    'feature': X_train.columns,
    'importance': tree_model.feature_importances_
}).sort_values('importance', ascending=False)


The top 10 most important features are listed below. We observe V11, E10, and V1 are the most important features in classification.

In [ ]:
importance_df.head(10)

,feature,importance
22,V11,0.192839
10,E10,0.121922
12,V1,0.070413
8,E8,0.065658
6,E6,0.065107
7,E7,0.063555
9,E9,0.056569
17,V6,0.053019
5,E5,0.050894
15,V4,0.032601


# Hyperparameter Tuning in Random Forest

Though we get very decent train, and test score, we still want to do hyperparameter tuning to make sure that the model is not having any overfitting problem. We use max_depth, max_leaf_nodes, and min_samples_split as hyperparameters.

In [ ]:
max_depth = [5,10,15,20,25,30,35,40]
for i in max_depth:
  tree_model = RandomForestClassifier(max_depth = i)
  tree_model.fit(X_train,y_train)
  train_tree = tree_model.predict(X_train)
  print(f"max_depth = {i}, train error:  ",1-tree_model.score(X_train,y_train),end = " ")
  val_tree = tree_model.predict(X_val)
  print(",  validation set error: ",1-tree_model.score(X_val,y_val) )

max_depth = 5, train error:   0.17110000000000003 ,  validation set error:  0.17547999999999997
max_depth = 10, train error:   0.09788333333333332 ,  validation set error:  0.10704000000000002
max_depth = 15, train error:   0.04448333333333332 ,  validation set error:  0.0686
max_depth = 20, train error:   0.01915 ,  validation set error:  0.05171999999999999
max_depth = 25, train error:   0.004750000000000032 ,  validation set error:  0.045399999999999996
max_depth = 30, train error:   0.0002166666666666428 ,  validation set error:  0.04308000000000001
max_depth = 35, train error:   1.666666666666483e-05 ,  validation set error:  0.04276000000000002
max_depth = 40, train error:   0.0 ,  validation set error:  0.042080000000000006


In [ ]:
max_leaf_nodes = [5,10,15,20,25,30,35,40,45,50,55,60,65,70,100,200,250,300,400,500,600,700,900,1000,2000,3000,5000,10000]
for i in max_leaf_nodes:
  tree_model =  RandomForestClassifier(max_leaf_nodes = i)
  tree_model.fit(X_train,y_train)
  train_tree = tree_model.predict(X_train)
  print(f"max_leaf_nodes = {i}, train error:  ",1-tree_model.score(X_train,y_train),end = " ")
  val_tree = tree_model.predict(X_val)
  print(",  validation set error: ",1-tree_model.score(X_val,y_val) )

max_leaf_nodes = 5, train error:   0.19305000000000005 ,  validation set error:  0.19767999999999997
max_leaf_nodes = 10, train error:   0.15861666666666663 ,  validation set error:  0.16347999999999996
max_leaf_nodes = 15, train error:   0.15433333333333332 ,  validation set error:  0.1582
max_leaf_nodes = 20, train error:   0.1472 ,  validation set error:  0.15139999999999998
max_leaf_nodes = 25, train error:   0.14428333333333332 ,  validation set error:  0.14871999999999996
max_leaf_nodes = 30, train error:   0.14196666666666669 ,  validation set error:  0.14603999999999995
max_leaf_nodes = 35, train error:   0.13726666666666665 ,  validation set error:  0.14083999999999997
max_leaf_nodes = 40, train error:   0.13570000000000004 ,  validation set error:  0.14080000000000004
max_leaf_nodes = 45, train error:   0.13021666666666665 ,  validation set error:  0.13419999999999999
max_leaf_nodes = 50, train error:   0.1296666666666667 ,  validation set error:  0.13495999999999997
max_leaf

In [ ]:
min_samples_split = [100,50,40,30,25,20,10,5,3,2]
for i in min_samples_split:
  tree_model = RandomForestClassifier(min_samples_split = i)
  tree_model.fit(X_train,y_train)
  train_tree = tree_model.predict(X_train)
  print(f"min_sample_split = {i}, train error:  ",1-tree_model.score(X_train,y_train),end = " ")
  val_tree = tree_model.predict(X_val)
  print(",  validation set error: ",1-tree_model.score(X_val,y_val) )

min_sample_split = 100, train error:   0.07361666666666666 ,  validation set error:  0.08664000000000005
min_sample_split = 50, train error:   0.05586666666666662 ,  validation set error:  0.07604
min_sample_split = 40, train error:   0.049900000000000055 ,  validation set error:  0.07016
min_sample_split = 30, train error:   0.042916666666666714 ,  validation set error:  0.06699999999999995
min_sample_split = 25, train error:   0.03891666666666671 ,  validation set error:  0.06384000000000001
min_sample_split = 20, train error:   0.03323333333333334 ,  validation set error:  0.060679999999999956
min_sample_split = 10, train error:   0.01846666666666663 ,  validation set error:  0.05259999999999998
min_sample_split = 5, train error:   0.005616666666666714 ,  validation set error:  0.045560000000000045
min_sample_split = 3, train error:   0.0003833333333332911 ,  validation set error:  0.043279999999999985
min_sample_split = 2, train error:   0.0 ,  validation set error:  0.042440000000

Thus, we choose max_depth = 10, min_samples_split = 50 and max_leaf_nodes= 10 for our optimal random forest model. 

# Prediction on Test Set using Optimized Decision Tree Model

In [ ]:
tree_model = RandomForestClassifier(random_state = 42,max_depth = 10, min_samples_split = 50, max_leaf_nodes= 10 )
tree_model.fit(X_train,y_train)
test_tree = tree_model.predict(test_set)


In [ ]:
accuracy_score(test_tree,sol_df['Prediction'])

0.8707133399536577

Finally, we achieve accuracy score 87.07% which is even higher than optimal logistic regression model.



# Conclusion
1. First of all, we download the dataset from kaggle and convert CSV file into Pandas DataFrame.

2. Then, we clean the dataset and keep the information that are required for our analysis.

3. After that, we split the data into training and validation set and their ratio is 80:20.

4. Since our dataset is very large and all the ML techniques implementaions are quite time consuming, we use a sample of 60000 for training from the splitted training set and 25000 data from new validation set.
5. Next, we perform Scaling to prepare our data for training.

6. After that, we apply Logistic Regression and Random Forest on our prepared data and we pergorm hyperparameter tuning.

7. Finally, we apply our optimal models on unseen test data and we achieve 84.1% accuracy for Logistic regression and 87.1%

# References


1.   https://scikit-learn.org/stable/modules/tree.html

2.  https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

3.  https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

4.  https://www.kaggle.com/willkoehrsen/introduction-to-manual-feature-engineering

5.  https://www.kaggle.com/willkoehrsen/intro-to-model-tuning-grid-and-random-search
 

In [89]:
jovian.commit()

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
Committed successfully! https://jovian.ai/aurobinda005/alertness


'https://jovian.ai/aurobinda005/alertness'